In [ ]:
import nltk
nltk.download('stopwords');
import time
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from collections import Counter
from config import *
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
import string
import unidecode
from wordcloud import WordCloud, ImageColorGenerator
import re
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words("english"))
ps = PorterStemmer()
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
 
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import multiprocessing
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from nltk import bigrams
import time
from nltk.util import ngrams

In [ ]:
with open("notebook/one.json", "rb") as f:
    lines = f.readlines()
    lines = [json.loads(str_) for str_ in lines]

In [ ]:
df_tweets = pd.DataFrame.from_records(lines)
df_tweets['h_field'] = df_tweets["entities"].apply(lambda x: x['hashtags'])
df_tweets['hashtags'] = df_tweets['h_field'].apply(lambda x: x[0]['text'] if x != [] else None)
df_tweets['retweeted_status'] = df_tweets['retweeted_status'].apply(lambda x: 1 if str(x)== 'nan' else 0)
df_tweets['id_tweet'] = df_tweets.index

data = df_tweets[df_tweets['retweeted_status'] == 1][['id_tweet', 'text']]
#data['id_tweet'] = str(data['id_tweet'])
data['join'] = data[['id_tweet', 'text']].apply(lambda x: '{}|{}'.format(x[0],x[1]), axis=1)

In [ ]:
def remove_links(text):
    if text:
        return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
    
    # In case there is no text
    return ""

def remove_hashtags(text):
    if text:
        return re.sub(r'#\w+ ?', '', text)
    
    return ""
def remove_accents(text):
    if text:
        return unidecode.unidecode(text)
        

    # In case there is no text
    return ""

def remove_punctuation_marks(text):
    if text:
        translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
        new_text = text.translate(translator)
        return " ".join(new_text.split())
        
    # In case there is no text
    return ""

def text_to_lower_case(text):
    if text:
        return text.lower()
    
    # In case there is no text
    return ""

def remove_emojis(text):
    if text:
        # TODO: Remove emojis (tip: search for encode - decode)
        returnString = ""
        for character in text:
            try:
                character.encode("ascii")
                returnString += character
            except UnicodeEncodeError:
                returnString += ''
        text = " ".join(returnString.split())
        
    return text

    # In case there is no text
    return ""

def remove_multiple_whitespaces(text):
    if text:
        return " ".join(text.split())

    # In case there is no text
    return ""

def remove_text_marks(text):
    if text:
        text = text.translate(str.maketrans('', '', string.punctuation))
        
        # TODO: replace *, ?, ... by spaces
        
        return text.strip()
    
    # In case there is no text
    return ""

def split_text_and_numbers(text):
    temp = re.compile("([a-zA-Z]+)([0-9]+)") 
    try:
        res = temp.match(text).groups()
        text = " ".join(res)
    except:
        text = text
    return text

def remove_alone_numbers(text):
    if text:
        text = ' '.join(filter(lambda word:word.replace('.','').isdigit()==False, text.split()))
        
        return text
    
    return ""

def remove_stopwords(text):
    text = text.split()
    return " ".join([x for x in text if x not in STOPWORDS])

def stemming(text):
    text = text.split()
    return " ".join([ps.stem(x) for x in text])

def clean_text(text):
    # Apply the different functions in order to clean the text
    text = remove_links(text)
    text = remove_hashtags(text)
    text = text_to_lower_case(text)
    text = remove_text_marks(text)
    text = remove_accents(text)
    text = remove_emojis(text)
    text = split_text_and_numbers(text)
    #text = remove_alone_numbers(text)
    text = remove_multiple_whitespaces(text)
    text = remove_punctuation_marks(text)
    text = remove_stopwords(text)
    text = stemming(text)
    
    # Return
    return text.split()

In [ ]:
queries = ['trump has covid', 'impact of coronavirus', 'how to face off covid',
           'masks and lockdowns don\'t work', 'how good is the vaccine?', 'vaccine of pfizer', 
          'vaccine 2021', 'no more masks', 'I think I have covid', 'what are covid symptoms']

In [ ]:
data['clean_text'] = data['text'].apply(lambda x: clean_text(x))

In [ ]:
sent = []
for line in data['clean_text']:
    sent.append(line)

In [ ]:
phrases = Phrases(sent, min_count = 30, progress_per= 10000)
bigram = Phraser(phrases)
sentences = bigram[sent]


In [ ]:
cores = multiprocessing.cpu_count()

w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

t = time.time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time.time() - t) / 60, 2)))

In [ ]:
w2v_model.init_sims(replace=True)

In [ ]:
w2v_model.wv.most_similar(positive=clean_text('trump'))

In [ ]:
from gensim.models import KeyedVectors
model=KeyedVectors


In [ ]:
keys = []
for line in sent:
    for i in line:
        if i not in keys:
            if i in w2v_model.wv.vocab:
                keys.append(i)


In [ ]:

embedding_clusters = []
word_clusters = []
for word in keys:
    embeddings = []
    words = []
    for similar_word, _ in w2v_model.most_similar(word, topn=30):
        words.append(similar_word)
        embeddings.append(w2v_model[similar_word])
    embedding_clusters.append(embeddings)
    word_clusters.append(words)

In [ ]:
from sklearn.manifold import TSNE
import numpy as np

embedding_clusters = np.array(embedding_clusters)
n, m, k = embedding_clusters.shape
tsne_model_en_2d = TSNE(perplexity=15, n_components=2, init='pca', n_iter=250, random_state=32)
embeddings_en_2d = np.array(tsne_model_en_2d.fit_transform(embedding_clusters.reshape(n * m, k))).reshape(n, m, 2)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
#


def tsne_plot_similar_words(title, labels, embedding_clusters, word_clusters, a, filename=None):
    plt.figure(figsize=(16, 9))
    colors = cm.rainbow(np.linspace(0, 1, len(labels)))
    for label, embeddings, words, color in zip(labels, embedding_clusters, word_clusters, colors):
        x = embeddings[:, 0]
        y = embeddings[:, 1]
        plt.scatter(x, y, c=color, alpha=a, label=label)
        #for i, word in enumerate(words):
            #plt.annotate(word, alpha=0.5, xy=(x[i], y[i]), xytext=(5, 2),
                         #textcoords='offset points', ha='right', va='bottom', size=8)
    plt.legend(loc=4)
    plt.title(title)
    plt.grid(True)
    if filename:
        plt.savefig(filename, format='png', dpi=150, bbox_inches='tight')
    plt.show()




In [ ]:
#% matplotlib inline
tsne_plot_similar_words('Similar words from Google News', keys, embeddings_en_2d, word_clusters, 0.7,
                        'similar_words.png')